##RAG챗봇 구현하기
‘2020_ 경제금융용어 700 선’ 이라는 PDF 파일을 이용하여 경제 금융 용어 챗봇을 만들어봅시다

In [ ]:
!pip install langchain_openai langchain_community pypdf chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.7 MB/s eta 0:00:

###1. 환경 설정 및 데이터세트 로드
 실습에 필요한 랭체인 도구들과 추후 데모를 위한 웹 UI 를 제공하는 gradio 패키지를 임포트합
니다.

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
import urllib.request
import gradio as gr

다음으로 os.environ['OPENAI_API_KEY']에 발급받은 OpenAI API 키 값을 입력합니다. 해당 코
드는 현재 실습 환경에 OpenAI API 키 값을 세팅한다는 의미입니다.

In [ ]:
os.environ['OPENAI_API_KEY'] = 'Openai_api_key'

이제 실습에 사용할 2020_ 경제금융용어 700 선 _ 게시.pdf 파일을 다운로드합니다. 다음 코드는 인터넷
의 코드 저장소에서 해당 파일을 직접 다운로드하는 코드입니다 (해당 파일은 한국은행 홈페이지에서도
다운로드할 수 있습니다).


In [ ]:
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf",
                           filename="2020_경제금융용어 700선_게시.pdf")

('2020_경제금융용어 700선_게시.pdf', <http.client.HTTPMessage at 0x7ff43d7f6450>)

이제 랭체인의 PyPDFLoader()를 통해 PDF 파일을 로드합니다. 이는 PDF 파일을 파이썬으로 읽고 싶
을 때 가장 많이 쓰는 대표적인 도구입니다. PyPDFLoader(파일명)을 실행하여loader라는 객체를
선언하고, 해당 객체를 통해 load_and_split()을 실행하면 PDF 를 여러 개의 청크로 분할한 문자열
리스트가 반환됩니다.

In [ ]:
loader = PyPDFLoader("2020_경제금융용어 700선_게시.pdf")
texts = loader.load_and_split()
print('문서의 수: ', len(texts))

문서의 수:  366


PDF 문서가 총 366 개의 텍스트 청크로 분할되었습니다. 임의로 15 번 청크를 출력해보겠습니다.

In [ ]:
texts[15]

Document(metadata={'producer': 'Smart PDF Imposition 1.0', 'creator': 'Smart PDF Imposition 1.0', 'creationdate': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'creationdate--text': 'January 31, 2018 16:21', 'moddate': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'source': '2020_경제금융용어 700선_게시.pdf', 'total_pages': 371, 'page': 19, 'page_label': '20'}, page_content='3\nㄱ \n신용을 모두 포괄하는 것으로 크게 가계대출과 판매신용으로 구분된다. 현재 가계신용통\n계는 2002년 말 잔액부터 분기별로 제공되고 있다. 가계신용통계는 우리나라 가계부문의 \n부채 규모 및 변동 등을 파악하는 데 널리 활용되고 있다 .\n가계처분가능소득 \n가계처분가능소득(PDI; Personal Disposable Income)은 가계가 맘대로 소비와 저축\n으로 처분할 수 있는 소득을 의미한다. 흔히 국민들의 생활수준을 파악해 볼 수 있는 \n지표로 1인당 GNI가 널리 쓰이고 있으나 국민총소득에는 가계 뿐 아니라 기업 금융기관 \n정부가 벌어 들인 소득이 모두 포함되어 있다. 따라서 기업과 금융기관 등이 가계부문 \n보다 더 많은 소득을 벌어 1인당 국민총소득(GNI)이 높아진 경우에는 가계가 느끼는 \n체감경기는 전체 경기와 괴리가 있게 된다. 1인당 가계총처분가능소득(PGDI; Personal \nGross Disposable Income)은 가계부문의 총처분가능소득을 연앙인구로 나누

출력 형식을 살펴보면 page_content에는 분할된 텍스트의 본문이 저장되어 있고, source에는
해당 본문의 원본 파일의 이름이 저장되어 있습니다. 기본적으로 랭체인에서 PyPDFLoader()로 객체를 선언하고, load_and_split()을 사용하는 경우에는 다음과 같은 형식을 따릅니다. ‑ Document(page_content=‘내용’, metadata={‘source’: 파일명, ‘page’: 페이지 번호}) 본문에만 접근하고 싶다면 각 청크 (문자열 원소) 에 .page_content를 붙여서 호출하면 됩니다.

In [ ]:
print(texts[15].page_content)

3
ㄱ 
신용을 모두 포괄하는 것으로 크게 가계대출과 판매신용으로 구분된다. 현재 가계신용통
계는 2002년 말 잔액부터 분기별로 제공되고 있다. 가계신용통계는 우리나라 가계부문의 
부채 규모 및 변동 등을 파악하는 데 널리 활용되고 있다 .
가계처분가능소득 
가계처분가능소득(PDI; Personal Disposable Income)은 가계가 맘대로 소비와 저축
으로 처분할 수 있는 소득을 의미한다. 흔히 국민들의 생활수준을 파악해 볼 수 있는 
지표로 1인당 GNI가 널리 쓰이고 있으나 국민총소득에는 가계 뿐 아니라 기업 금융기관 
정부가 벌어 들인 소득이 모두 포함되어 있다. 따라서 기업과 금융기관 등이 가계부문 
보다 더 많은 소득을 벌어 1인당 국민총소득(GNI)이 높아진 경우에는 가계가 느끼는 
체감경기는 전체 경기와 괴리가 있게 된다. 1인당 가계총처분가능소득(PGDI; Personal 
Gross Disposable Income)은 가계부문의 총처분가능소득을 연앙인구로 나누어 계산한 
지표로 가계의 구매력을 가장 정확히 가늠해 볼 수 있는 소득지표이다 . 
 연관검색어 : 국민총소득 (GNI)
가교은행
청산 대상 금융기관의 자산, 부채를 임시로 넘겨받아 예금, 출금 등의 업무를 대행하고 
합병, 채권채무관계 조정 등 후속조치를 수행하는 은행을 말한다. 금융기관이 파산한 경우 
예금보험제도는 청산, 매각, 자산부채승계, 가교은행을 통한 인수 등의 수단을 통해 부실 
금융기관을 처리한다. 이 중 가교은행을 통한 인수방식은 파산은행의 처리에 상당한 시간이 
소요될 것으로 예상할 경우 사용되며 새로운 은행을 설립하여 자산, 부채를 포괄승계하도록 
하고, 인수 희망자를 물색하는 등의 조치를 취하게 된다. 가교은행을 이용한 처리방식은 
금융기관 파산에 따른 충격을 완화하고 시간적인 여유를 확보할 수 있는 것이 장점이다. 
임무를 마치면 정리대상 금융기관과 마찬가지로 없어지는 한시적 기관이다.
가동률
생산능력 대비 생산실적의 백분율(생산실적/생산능력×100)로

앞의 출력 결과에서 page_content의 값만 추출한 것을 볼 수 있습니다.

###2. 데이터 세트 전처리
이제 챗봇을 만들기 전에 챗봇에 필요 없는 정보들을 삭제하기 위해 각 청크를 확인해야 합니다. 먼저 가
장 첫 번째 청크인 0 번 청크를 출력해봅시다.

In [ ]:
print(texts[0].page_content)

iii
찾아보기
한국은행은 국민들이 경제 및 금융에 대한 이해도를 높이고 경제에 관한 합리적인 
의사결정 능력을 키울 수 있도록 현장 경제교육, 온라인 경제교육, 경제교육 콘텐츠 
개발 등 대국민 경제교육을 다양하게 수행해 오고 있습니다 . 
이의 일환으로 2018년 
 경제금융용어 700선
 을 발간하였는데 그간 동 책자에 대한 
수요가 꾸준히 늘어남에 따라 이번에 추가로 발간하게 되었습니다 .
지난번 내용과 같이 통화정책, 실물경제, 금융안정, 지급결제 등 한국은행 주요 
업무를 이해하는데 필요한 전문 용어와 경제·금융 흐름을 이해하는데 도움이 되는 
시사 경제금융 용어들을 수록하였습니다. 용어해설은 개념과 도입 배경, 의미, 적용 
사례 등을 담아 쉽게 이해할 수 있도록 하였습니다. 또한 e-book으로도 제작하여 
독자들의 편의성과 가독성을 높였습니다 . 
코로나 19로 이전에 경험하지 못한 경제위기를 겪으면서 경제·금융 현상에 대한 
일반인들의 관심도가 이전에 비해 높아졌습니다. 아무쪼록 이 책자가 한국은행의 주요 
정책과 국내외 경제를 이해하는 데 조금이나마 도움이 되기를 바라겠습니다. 끝으로 
경제금융용어 700선
  집필에 참여해주신 경제교육실 교수님들과 용어 집필에 좋은 
의견을 주신 관련부서 여러분께 감사드립니다 .
2020년 8월
한국은행 경제교육실장 박 철 원
머 리 말


0번 청크는 해당 PDF 파일 머리말에 해당됩니다. 챗봇 개발자의 판단에 따라 다를 수 있지만 해당 정보가 챗봇이 제공할 정보가 아니라고 판단되면 해당 청크를 삭제하는 것을 고려해야 합니다. 이제 5번 청크를 출력해 봅시다.

In [ ]:
print(texts[5].page_content)

vi
경제금융용어 700선
ㅂ
바이오인증 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・118
바젤은행감독위원회/
바젤위원회(BCBS) ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・118
발행시장 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・119
발행중지화폐/유통정지화폐 ・・・・・・・・・・・・・・・120
방카슈랑스 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・120
배당할인모형 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・121
밴드웨건효과 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・121
뱅 크 런 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・122
범위의 경제 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・122
법률리스크 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・122
베블런효과 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・123
변동금리 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・123
변동금리부채권(FRN) ・ ・ ・・・・・・・・・・・・・・・・・・・・・・・123
보기화폐(견양화폐) ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・124
보완자본(Tier 2) ・・・・・・・・・・・

5번 청크는 PDF 파일에서의 목차에 해당됩니다. 예제 챗봇의 목적이 경케 금융 용어를 설명사는 것으로 챗봇에 필요한 정보에는 해당되지 않습니다. 이제 12번 청크를 출력해 봅시다.

In [ ]:
print(texts[12].page_content)

xiii
찾아보기❙
환매조건부매매/RP/Repo ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・331
환 어 음 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・332
환율조작국 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・332
환전영업자(환전상) ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・333
환차손/환차익 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・333
황금낙하산 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・334
회 사 채 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・334
회수의문 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・335
후방연쇄효과 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・335
후순위금융채 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・335
후행종합지수 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・336
ABC
B e y o n d  G D P ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・337
BIS 자기자본비율 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・338
CAMEL-R시스템/ROCA시스템/
C A C R E L 시 스 템 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・338
C L S 은 행 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・339
C M O ・・・・・・・・・・・・・・・・

마찬가지로 목차에 해당됩니다. 13 번 청크를 출력해봅시다.

In [ ]:
print(texts[13].page_content)

1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


13 번 청크부터는 경제 금융 용어 설명이 출력됩니다. 앞에 몇 개의 청크를 출력해본 결과, 0 번 청크는
머리말, 12 번 청크까지는 목차, 13 번 청크부터 금융 용어를 설명하는 청크임을 확인했습니다. 다말해 용어를 검색하는 데는 0 번 청크부터 12 번 청크는 필요하지 않습니다. 기존의 0 번 청크와 12 번
청크까지 제거해보겠습니다. 이러한 판단은 원본 PDF 파일과 PyPDFLoader()로 객체를 선언하고,
load_and_split()을 수행하여 얻은 청크들을 보고 챗봇의 개발자가 판단해야 합니다. 챗봇의 답변
에 불필요한 정보들을 제거해야만 챗봇의 잘못된 답변을 막을 수 있습니다. 마찬가지로 목차에 해당됩니
다. 다음 코드는 13 번 청크부터 시작하도록 수정하여 texts를 재저장한다는 의미입니다.

In [ ]:
texts = texts[13:]
print('줄어든 청크의 갯수: ', len(texts))

줄어든 청크의 갯수:  353


그럼 0 번 청크를 출력해보겠습니다. 전처리가 제대로 되었다면 이전의 13 번 청크가 현재의 0 번 청크가
되어야 합니다.

In [ ]:
print(texts[0].page_content)

1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


보다시피 전처리가 정상적으로 수행되었습니다. 이번에는 뒤에 있는 청크들을 확인하기 위해 마지막 청
크를 출력해보겠습니다. 파이썬 리스트에서는 ‑1 이 마지막 원소를 의미합니다.


In [ ]:
print(texts[-1].page_content)

경제금융용어 700선
발행인 이주열
편집인 박철원
발행처 한국은행(www.bok.or.kr)
서울특별시 중구 세종대로 67(태평로 2가)
발행일 2020년 8월 3일
인  쇄 ㈜제일프린테크 
 본 자료는 한국은행 홈페이지(http://www.bok.or.kr>경제교육>온라인학습>일반인) 에서 
다운로드 받으실 수 있습니다.
 이 책자에 대한 문의는 한국은행 경제교육실 경제교육기획팀(02-759-5618)으로 연락 
하여 주시기 바랍니다.
 
 경제금융용어 700선
 은 
① 정부간행물판매센터(☎ 02-734-6818, http://www.gpcbooks.co.kr) 
② 한국경제서적(☎ 02-737-7498)
③ 경제서적(☎ 02-736-0640, http://kj-book.co.kr) 또는 주요 서점 등에서 매권당 
8,000원에 판매하고 있습니다.
ISBN 979-11-5538-393-3 03320


해당 청크는 이 문서의 맺음말에 해당되므로 해당 데이터도 불필요합니다. 마지막에서 두 번째 청크를 출
력해보

In [ ]:
print('마지막에서 두번째 청크: ',texts[-2].page_content)

마지막에서 두번째 청크:  352
경제금융용어 700선
와 관련된 다양한 부수업무를 하는 회사를 말한다. 가령 신용카드 거래가 발생하면 
VAN사업자는 거래 승인을 위해 신용카드사에 거래내역을 전송하고 승인 및 조회결과를 
수신한 후 이를 가맹점에 전송한다. 또한 VAN사업자는 가맹점이 신용카드사에 전표를 
제출하여 대금을 청구하는 업무를 대신하기 위하여 가맹점을 방문하여 전표실물을 수거
하거나 전표 데이터를 수집한다 . 
 연관검색어 : 지급결제시스템, 전자금융
VaR(Value at Risk) 
주어진 신뢰수준 하에서 일정 기간 동안 발생할 수 있는 ‘최대 손실금액’으로 금융기관
의 잠재적인 손실을 측정하는 지표이다. 예를 들어 목표기간 1년, 신뢰수준 95% 에서 
산출된 VaR가 10억 원이라면 이는 1년 동안 발생할 수 있는 손실금액이 10억 원보다 
작을 확률이 95%라는 것을 의미한다 . 
 연관검색어 : 예상손실
VIX
미국 주식시장의 단기 변동성에 대한 시장의 기대치를 나타내는 지수로 시카고옵션거
래소(CBOE)에서 제공되며, 정식명칭은 CBOE Volatility Index이다. VIX는 향후 30 일 
동안의 S&P 500 지수의 변동성에 대한 시장의 기대치로서, 지수의 변동성이 클 것으로 
예상될 경우 옵션가격이 높아지는 점에 착안하여 CBOE에 상장된 다양한 행사가격의 
S&P 500 지수 옵션들의 가격을 활용하여 산출된다. VIX는 일반적으로 기초자산 가격과 
음(-)의 상관관계가 있다. 예를 들어 주가지수가 상승할 때 하락하고 주가지수가 하락할 
때는 상승한다. 이에 따라 VIX의 상승은 투자자들의 불안심리가 증대하는 것을 의미하므
로 공포지수라고도 부른다. 1993년 Robert E. Whaley 교수의 논문에서 처음 소개된 
이후 시장의 변동성과 투자자들의 심리를 나타내주는 주요한 지표로 널리 활용되고 
있다. 이 외의 대표적인 주식시장 변동성 지수로는 유럽의 VSTOXX 등이 있다. VSTOXX
는 유럽의 대표 주가지수인 Eur

마지막에서 두번째 문서는 경제 금융용어에 해당하므로 삭제해서는 안됩니다. 마지막 데이터를 제거한후 정상 적으로 제거 되었는지 확인하기 위해서 청크의 수를 출력합니다.

In [ ]:
texts = texts[:-1]
print('마지막 데이터 제거 후 청크의 수: ', len(texts))

마지막 데이터 제거 후 청크의 수:  352


그럼 지금까지 설명한 과정을 정리해봅시다. 랭체인의 PyPDFLoader()를 통해 PDF 파일을 읽고, load_and_split()을 수행하여 PDF 파일을 다수의 청크로 분리했습니다. 해당 PDF 파일의 경우 문서의 앞쪽 페이지와 뒤쪽 페이지들은 머리말, 목차, 맺음말 등으로 구성되어 있는데, 이는 챗봇이 답해야 하는 요소가 아니므로 PDF 파일을 직접 열거나 중간 중간 출력해 보면서 이렇게 불필요한 요소들을 제거했습니다.


###3. 임베딩 및 벡터 데이터베이스 적재
이제 금융 용어에 대한 설명으로만 구성된 352 개의 청크를 모두 임베딩하여 벡터 데이터베이스에 적재
해보겠습니다. OpenAI 의 Embedding API 를 이용하여 텍스트를 임베딩하고, 코사인 유사도를 통해 유사
한 텍스트를 가져오는 실습을 진행한 바 있습니다. 여기서도 OpenAI 의 Embedding API 를 사용하겠습니
다. 크로마 DB 는 이 과정을 기능별로 이미 구현하여 사용자가 벡터를 좀 더 쉽게 다룰 수 있도록 도와주는
편리한 벡터 응용 도구입니다.
Chroma.from_documents()를 통해 벡터 데이터베이스 객체인 vectordb를 선언합니다. 이때
documents에는 벡터화의 단위가 될 텍스트 리스트를 매개변수로 사용하고, embedding 에는 어떤 종
류의 임베딩을 사용할 것인지를 지정합니다.

In [ ]:
embedding = OpenAIEmbeddings(chunk_size= 100)

vectordb = Chroma.from_documents(
    documents = texts,
    embedding=embedding
)

/tmp/ipython-input-18-2106116231.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(chunk_size= 100)


vectordb를 선언하고 나면 ._collection 다음에 온점을 찍고 다양한 함수들을 사용할 수 있습니다. 예를 들어 count()는 현재 저장된 청크 또는 벡터 개수를 의미합니다.

In [ ]:
print(vectordb._collection.count())

352


앞서 확인했던 청크 개수인 352 개와 동일합니다. 기본적으로 _collection.get()은 벡터 데이터베
이스 객체인 vectordb에 저장된 값들을 볼 수 있는 기능을 갖고 있습니다. 어떤 값들을 호출할 수 있는
지 확인해봅시다.


In [ ]:
for key in vectordb._collection.get():
  print(key)

ids
embeddings
documents
uris
included
data
metadatas


보다시피 ids, embeddings, metadatas, documents를 호출할 수 있습니다. vectordb에 저장
된 기존 청크들을 보고 싶다면['documents']를 통해 불러올 수 있습니다. vectordb로부터 청크들
을 다시 로드하고, 청크의 개수와 임의로 0 번 청크를 출력해보겠습니다.


In [ ]:
documents = vectordb._collection.get()['documents']
print('청크의 수: ', len(documents))
print('---'*15)
print(documents[0])

청크의 수:  352
---------------------------------------------
1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


이때 0 번 청크는 수많은 352 개의 청크 중 임의로 선택되어 출력된 청크로 실제 순서가 보장되지 않으
므로 위에서 선택되어 출력되는 청크는 위 결과와 다를 수 있습니다. 임베딩 벡터의 값은 기본적으로는
제공하지 않기 때문에 임베딩 벡터의 값도 확인하고 싶다면 get()을 호출할 때 내부에 include=['
embeddings']를 기재해야 합니다. 그 후 ['embeddings']를 통해 호출할 수 있습니다. 다음 코드
는 352 개 청크의 임베딩 벡터를 불러와 embeddings에 저장합니다. 그 후 임베딩 벡터의 개수를 출력
하여 352 개로 일치하는지 확인합니다.

In [ ]:
embeddings = vectordb._collection.get(include=['embeddings'])['embeddings']
print('임베딩 벡터의 개수: ', len(embeddings))

임베딩 벡터의 개수:  352


0 번 청크의 임베딩 벡터의 값과 0 번 청크의 임베딩 벡터 값의 길이를 출력해보겠습니다

In [ ]:
print('0번 청크의 임베딩 값: ', embeddings[0])
print('0번 청크의 임베딩 값 길이: ', len(embeddings[0]))

0번 청크의 임베딩 값:  [-0.0077088  -0.0075166   0.014182   ... -0.01861645 -0.00669286
 -0.00903364]
0번 청크의 임베딩 값 길이:  1536


이때 0 번 청크의 임베딩 값은 수많은 352 개의 청크의 임베딩 값 중 임의로 선택되어 출력된 것으로 실제
순서가 보장되지 않으므로 임베딩 길이는 동일하더라도 임베딩 값은 위 결과와 다를 수 있습니다. OpenAI
의 Embedding API 을 사용하는 OpenAIEmbeddings()를 사용하면 각 청크는 1,536 개의 숫자를 가
진 벡터로 변환됩니다. 이번에는 metadatas를 호출해봅시다. metadatas는 각 청크의 출처를 의미
합니다.


In [ ]:
metadatas = vectordb._collection.get()['metadatas']
print('metadatas의 개수: ', len(metadatas))
print('0번 청크의 출처: ', metadatas[0])

metadatas의 개수:  352
0번 청크의 출처:  {'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'source': '2020_경제금융용어 700선_게시.pdf', 'subject': 'CoreImposition PDF 1.0 Output', 'creationdate--text': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'moddate': 'January 31, 2018 16:21', 'page': 17, 'producer': 'Smart PDF Imposition 1.0', 'creationdate': 'January 31, 2018 16:21', 'total_pages': 371, 'author': 'INSUNG DATA INC.', 'creator': 'Smart PDF Imposition 1.0', 'page_label': '18'}


이때 0 번 청크의 출처는 수많은 352 개의 청크 값 중 임의로 선택되어 출력된 것으로 실제 순서가
보장되지 않으므로 0 번 청크의 출처는 위 결과와 다를 수 있습니다. 벡터 도구 객체를 선언하고
나면 as_retriever()를 통해 입력된 텍스트로부터 유사한 텍스트를 찾아주는 검색기 객체인
retriever를 선언할 수 있습니다. 그 후에 get_relevant_documents(입력 텍스트)를 통해
입력된 텍스트와 유사한 청크들을 찾아서 반환합니다. 이러한 과정을 통해 벡터의 유사도를 구하는
과정을 별도의 추가 구현 없이 손쉽게 사용할 수 있습니다. 검색기 객체인 retriever를 선언할 때
as_retriever()에 search_kwargs={"k": 숫자}를 전달하면 입력 텍스트로부터 유사한 청
크를 검색할 때 유사도를 기준으로 몇 번째 순위까지를 반환할 것인지를 지정할 수 있습니다. 예를 들어 k
의 값을 2 로 지정하면 유사도를 기준으로 상위 2 개의 청크를 반환합니다. 상위 2 개의 청크만 반환하도
록 검색기 객체인 retriever 를 선언하고, “비트코인이 궁금해” 라는 입력 텍스트로 검색해보겠습니다.

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k":2})

docs = retriever.get_relevant_documents("비트코인이 궁금해")
print('유사 문서 개수 : ', len(docs))
print('_-_'*30)
print("첫번째 유사 문서: ", docs[0])
print('_-_'*30)
print("두번째 유사 문서: ", docs[1])

/tmp/ipython-input-25-1343588916.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("비트코인이 궁금해")


유사 문서 개수 :  2
_-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-_
첫번째 유사 문서:  page_content='139
ㅂ 
비트코인
비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템이다. 비트코인 시스템
은 중앙 저장소 또는 단일 관리자가 없기 때문에 최초의 탈중앙화된 디지털통화라고 불린다. 
이는 사토시 나카모토라는 사람(집단)에 의해 만들어져서 2009년 개방형 소프트웨어로 
배포되었다. 이 시스템은 공유형(peer-to-peer)이며, 거래는 중개자 없이 블록체인 소프트
웨어를 이용하는 참여자(nodes) 사이에 직접 이뤄진다. 이런 거래들은 공유(P2P) 네트워크
상 참여자의 작업증명(proof-of-work)을 통해 검증되고 공개된 분산원장인 블록체인에 
기록된다. 승인된 거래의 새 그룹인 한 블록은 대략 10분에 한 번씩 만들어져서 블록체인에 
추가되고 신속하게 모든 참여자에게 보내어진다. 비트코인은 대규모 전기를 사용하는 
컴퓨터 처리 능력(power)을 활용해서 이뤄지는 기록보관 서비스인 채굴(mining)에 대한 
보상으로 발행되고 있으며 다른 통화･상품･용역 등과 교환되어가고 있다. 중앙은행이 
발행한 법정화폐가 아닌 비트코인은 비트코인 플랫폼에서 거래되며 투자대상으로서도 
관심을 받고 있다. 하지만 급등한 가격 및 심한 변동 폭으로 거품논란이 크다. 또한 익명성으
로 자금세탁 등 불법거래에 악용될 소지가 많아 중국 등 일부 국가에서 비트코인 등 가상통
화의 거래를 규제하고 있다. 일본의 경우 비트코인의 거래이익에 대해 세금을 부과한다. 
비트코인은 추가되는 한 블록당 새로 12.5비트코인을 보상하는데(2016.7월 현재), 21만개
가 채굴될 때(대략 4년)마다 그 보상이 반으로 줄어든다. 비트코인의 총량은 21백만개이며 
2140년경 모두 채굴될 것으로 전망된다. 비트코인은 그 시스템의 설계시 그 수

검색 결과를 보면 첫 번째 청크로 비트코인 용어에 대한 청크가, 두 번째 청크로 블록 체인 용어에 대한 청
크가 검색되어 본문 중간에 비트코인이라는 단어가 언급되고

###4. 프롬프트 템플릿, LLM, 리트리버의 연결
이제 이러한 검색기를 GPT‑4 와 연결하는 작업을 진행해보겠습니다. 랭체인을 통해 GPT‑4 를 연결
할 때 프롬프트를 생성하는 방법으로 프롬프트 템플릿이 존재합니다. 프롬프트 템플릿을 이용하여
GPT‑4 에 사용할 프롬프트를 만들어 보겠습니다. 우선 해당 챗봇에게 한국은행에서 만든 금융 용어를
설명하는 챗봇으로 이름은 ‘금융쟁이’ 이며 안상준 개발자가 제작했고 주어진 검색 결과를 바탕으로
만 답변하라는 지시문을 작성합니다. 그런 다음, {context}와 {question}이라는 두 개의 변수를
할당합니다. {context}는 앞으로 사용자의 입력 텍스트로부터 검색된 결과들이 들어갈 자리이며,
{question}은 사용자의 입력 텍스트가 들어갈 자리입니다. 예를 들어 앞의 코드에서는 ‘비트코인이
궁금해’ 라는 입력 텍스트를 검색하여 두 개의 검색 결과를 얻었었는데, 다음의 템플릿에 따르면 ‘비트
코인이 궁금해’ 는 {question}에 위치하게 되며, {context}의 위치에는 2 개의 검색 결과가 들어가
게 됩니다. 이렇게 만들어진 프롬프트 문자열을 PromptTemplate.from_template()의 입력으
로 사용하여 프롬프트 템플릿 객체인 prompt를 만

In [ ]:
template = """당신은 한국은행에서 만든 금융 용어를 설명해주는 금융쟁이입니다.
안상준 개발자가 만들었습니다. 주어진 검색 결과를 바탕으로 답변하세요.
검색 결과에 없는 내용이라면 답변할 수 없다고 하세요. 반말로 친근하게 답변하세요.
{context}

Question: {question}
Answer:
"""

prompt = PromptTemplate.from_template(template)

이제 GPT‑4 와 연결하기 위해서 ChatOpenAI()를 이용하여 LLM 객체에 해당하는 llm을 만듭니다. 수
많은 GPT‑4 모델 중 하나를 선택하여 model_name의 값으로 결정하면 되는데, 여기서는 GPT‑4 버전 중
에서 답변 속도가 빠른 gpt-4o를 선택했습니다. temperature는 GPT‑4 의 답변에 무작위성을 주는
값으로서 이 값이 높으면 예측하지 못한 답변을 주기에 창의성에 비유하기도 합니다. 여기서는 값을 0 으
로 주어 의외의 답변을 할 가능성을 줄입니다

In [ ]:
llm = ChatOpenAI(model_name = 'gpt-4o', temperature=0)

이제 LLM 객체인 llm, 프롬프트 템플릿 객체인 prompt, 검색기 객체인 retriever를 연결합니다. 이
3 개의 객체를 연결하는 도구로 랭체인에서는 RetrievalQA.from_chain_type()을 지원합니다.
RetrievalQA.from_chain_type()에 각각의 객체를 연결하는 Chain 객체인 qa_chain을 만듭
니다.

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type_kwargs = {"prompt": prompt},
    retriever = retriever,
    return_source_documents =True
)

이제 qa_chain을 통해 사용자의 입력으로부터 서울 청년 정책과 관련된 챗봇의 답변을 얻을 수 있습니다. 이번에는 ‘디커플링이란 무엇인가?’ 라는 텍스트를 입력하여 qa_chain의 반환 결과를 확인해봅시다.


In [ ]:
input_text = "디커플링이란 무엇인가?"
chatbot_response = qa_chain(input_text)
print(chatbot_response)

/tmp/ipython-input-29-440151274.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chatbot_response = qa_chain(input_text)


{'query': '디커플링이란 무엇인가?', 'result': '디커플링은 탈동조화라고도 하는데, 어떤 나라나 지역의 경제가 인접한 다른 국가나 전반적인 세계 경제의 흐름과는 다른 모습을 보이는 현상을 말해. 예를 들어, 금융위기 이후 신흥국가나 유로지역 국가들이 미국 경제와 다른 모습을 보이는 경우가 있어. 주가나 금리, 환율 등 일부 경제 변수의 흐름이 국가 간 또는 특정 국가 내에서 서로 다른 흐름을 보이는 것도 디커플링이라고 할 수 있어. 반대로, 경제가 다른 국가나 세계 경제 흐름과 유사한 흐름을 보이는 것을 커플링이라고 해.', 'source_documents': [Document(metadata={'creationdate': 'January 31, 2018 16:21', 'total_pages': 371, 'moddate': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'page_label': '115', 'producer': 'Smart PDF Imposition 1.0', 'page': 114, 'source': '2020_경제금융용어 700선_게시.pdf', 'subject': 'CoreImposition PDF 1.0 Output', 'moddate--text': 'January 31, 2018 16:21', 'creator': 'Smart PDF Imposition 1.0', 'creationdate--text': 'January 31, 2018 16:21'}, page_content='98\n경제금융용어 700선\n가계와 기업이 소비와 투자를 미룸으로써 수요의 위축을 초래하여 디플레이션 압력을 \n보다 크게 할 수도 있다. 이 같은 상황에서는 경제주체들의 물가상승률 예상치가 더 \n낮아지지 않도록 사전에 통화 ･재정정책 면에서 적극적으로 대응하는 것이 중요하다 .\n 연관검색어 : 디플레이션,  통화정책\n디커

qa_chain의 결과를 chatbot_response 변수에 저장하여 출력해 보면 query, result, source_documents라는 세 개의 키 값을 갖고 있습니다. 이 가운데 source_documents는 앞서 PyPDFLoader()로 객체를 선언하고, load_and_split()을 수행하여 얻은 청크들입니다. 이들 각각은 page_content와 meta_data 값을 가집니다. query는 현재 입력된 사용자의 질문을 의미합니다. 이때 qa_chain은 앞서 구현한 retriever를 통해 내부적으로 ‘디 커플링이란무엇인가? ’와
가장 유사도가 높은 2 개의 청크를 찾아냅니다. metadata의 source 값에 따르면 해당 텍스트는 원본 ‘ 2020_경제금융용어 700선_게시.pdf’ 파일에서 각각 113 페이지와 114 페이지에 있는 청크에 해당 합니다. 챗봇은 이 두개의 본문 (여기서는 page_content의 값)과 앞서 작성한 프롬프트를 활용해 답변을 작성합니다. 해당 챗봇의 답변은 결과적으로 result의 값에 해당합니다. 현재 chatbot_response의 출력 결과에는 여러 개의 키가 포함되어 있어 복잡합니다. 따라서 chatbot_response로부터 챗봇의 답변에 해당하는 result 부분을 출력하는 get_chatbot_response() 함수를 구현합니다.

In [ ]:
def get_chatbot_response(input_text):
  chatbot_response = qa_chain(input_text)
  return chatbot_response['result']

In [ ]:
input_text = '너는 뭘하는 챗봇이니?'
result = get_chatbot_response(input_text)
result

'나는 한국은행에서 만든 금융 용어를 설명해주는 금융쟁이야. 안상준 개발자가 만들었고, 주어진 검색 결과를 바탕으로 답변해줄 수 있어. 검색 결과에 없는 내용이라면 답변할 수 없어. 궁금한 게 있으면 물어봐!'

‘비트코인에 대해서 궁금하당 ~’ 이라는 입력에 대한 챗봇의 답변을 확인해 보겠습니다

In [ ]:
input_text = '비트코인에 대해서 궁금하당~'
result = get_chatbot_response(input_text)
result

'비트코인은 가상통화이자 디지털 지급 시스템으로, 중앙 저장소나 단일 관리자가 없는 최초의 탈중앙화된 디지털 통화야. 사토시 나카모토라는 사람(또는 집단)이 2009년에 만들었고, 블록체인 기술을 기반으로 하고 있어. 거래는 중개자 없이 참여자들 사이에서 직접 이루어지고, 작업증명을 통해 검증된 후 블록체인에 기록돼. 비트코인은 채굴이라는 과정을 통해 발행되고, 다른 통화나 상품, 용역과 교환될 수 있어. 하지만 가격 변동이 심하고, 익명성 때문에 불법 거래에 악용될 가능성도 있어서 일부 국가에서는 규제를 하고 있어. 비트코인의 총량은 2100만 개로 한정되어 있고, 2140년경 모두 채굴될 것으로 예상돼.'

###5. 그라디오 웹 인터페이스
이번에는 Gradio 로 챗봇 UI 를 구현해보겠습니다. Gradio 는 AI 모델을 웹 형태로 배포할 수 있게 돕는 파
이썬 라이브러리입니다. 실제로 코드 몇 줄만으로 웹 기반의 인터페이스를 구현할 수 있습니다. Gradio
공식 문서에서는 Gradio 로 챗봇을 구현할 수 있도록 기본 코드를 제공합니다.

- Gradio 로 챗봇을 만드는 방법에 관한 공식 문서: https://www.gradio.app/guides/creating‑a‑chatbot

In [ ]:
print(gr.__version__)

5.38.2


In [ ]:
# 인 터 페 이 스 생 성
with gr.Blocks() as demo:
  chatbot = gr.Chatbot(label="경제 금융용어 챗봇") # 챗봇 레이블을 좌측 상단에 구성
  msg = gr.Textbox(placeholder="질문을 입력하세요") # 하단의 채팅창 레이블
  clear = gr.Button("대화 초기화") # 대화 초기화 버튼

  # 챗봇의 답변을 처리하는 함수
  def respond(message, chat_history):
    result = qa_chain(message)
    bot_message = result['result']

    # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가
    chat_history.append((message, bot_message))
    return "", chat_history

  # 사용자의 입력을 제출(submit)하면 respond 함수가 호출
  msg.submit(fn=respond, inputs=[msg, chatbot], outputs=[msg, chatbot])

  # '초기화' 버튼을 클릭 하면 채팅 기록을 초기화
  clear.click(lambda: None, None, chatbot, queue=False)

# 인 터 페 이 스 실 행
demo.launch(debug=True)


/tmp/ipython-input-35-505011714.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="경제 금융용어 챗봇") # 챗봇 레이블을 좌측 상단에 구성


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b0f1cb195d4876c4a1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


여기서 respond() 함수는 챗봇의 답변을 관리하는 함수입니다. 공식 문서에서는 respond() 함수에
사용자의입력변수인message부터챗봇UI화면에서보여줄챗봇의답변을만들어chat_history라
는 변수로 반환하라고 안내하고 있습니다. 여기서는 qa_chain()을 통해 반환된 값 중 챗봇의 실제 답변
인 result와 답변의 출처를 표기하는 metadata 내의 source 값을 함께 정리하여 chat_history
로 반환합니다. 그리고 gr.Chatbot(), gr.Textbox(), gr.Buttion()은 각각 챗봇 UI 화면에
표시될 주요 구성 요소에 표기할 텍스트 레이블입니다. gr.Chatbot()은 코드 실행 후 화면에 표시될
레이블이며, gr.Textbox()는 챗봇의 대화를 전송하기 위한 버튼의 레이블, gr.Buttion()은 대화
이력을 초기화하는 버튼의 레이블입니다. 위 코드를 실행하면 다음과 같은 챗봇 UI 가 화면에 나타납니
다. 챗봇에 원하는 질문을 입력하면 챗봇의 답변이 반환됩니다. 다음은 비트 코인에 대해 간단히 답변해
달라고 요청했을 때의 답변입니다.

이렇게 해서 이번 실습에서는 복잡한 PDF 파일을 청크로 나누고 불필요한 청크들을 제거한 후에 벡터 데
이터베이스와 GPT‑4o 와 연동하여 빠르고 쉽게 챗봇을 만들어보았습니다. 여러분도 다양한 정보를 담고
있는 PDF 를 사용하여 여러분만의 RAG 챗봇을 만들어보기 바랍니다.